In [3]:
import pandas as pd
import numpy as np

# Cargar la base de datos
DF_costos = pd.read_excel('Base costos minera centinela.xlsx', sheet_name='Costos')

# Suponiendo que la primera columna es un identificador o fecha, la excluimos
DF_costos_numericos = DF_costos.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')

# Filtrar los datos eliminando filas con solo NaN, pero manteniendo todas las columnas
DF_costos_filtrados_sin_cero = DF_costos_numericos.replace(0, np.nan).dropna(how='all', axis=0)

# Calcular la variabilidad (std/mean) de cada clase
variabilidad_filtrada_sin_cero = DF_costos_filtrados_sin_cero.std() / DF_costos_filtrados_sin_cero.mean()

# Crear DataFrame con estadísticas
DF_estadisticas_costos = DF_costos_numericos.describe().transpose()[['mean', 'min', 'max']].copy()
DF_moda = DF_costos_numericos.mode().transpose()[0]
DF_estadisticas_costos['moda'] = DF_moda
DF_desviacion_estandar = DF_costos_numericos.std()
DF_estadisticas_costos['desviacion_estandar'] = DF_desviacion_estandar
DF_estadisticas_costos['coef_variacion'] = variabilidad_filtrada_sin_cero

# Lógica para clasificación:
def clasificar_costo(row):
    if row['mean'] == row['min'] == row['max']:
        if row['mean'] == 0:
            return'Sin Registros'
        else:
            return'Fijo'
    elif row['coef_variacion'] > umbral:
        return'Variable'
    else:
         return'Fijo'

umbral = 1.0# Umbral para clasificación# Aplicar la lógica de clasificación
DF_estadisticas_costos['Clasificacion'] = DF_estadisticas_costos.apply(clasificar_costo, axis=1)

# Resumen de la clasificación
conteo_clasificados = DF_estadisticas_costos['Clasificacion'].value_counts()
DF_resumen = pd.DataFrame({
    'Categoria': ['Costos Fijos', 'Costos Variables', 'Sin Registros'],
    'Cantidad': [conteo_clasificados.get('Fijo', 0), conteo_clasificados.get('Variable', 0), conteo_clasificados.get('Sin Registros', 0)]
})

# Exportar a un solo archivo Excel con dos hojas
ruta_archivo_salida = r'C:\Users\Administrator\Documents\GitHub\Hackathon2-2024\1. Desarollo\Resultados\Analisis costos.xlsx'
with pd.ExcelWriter(ruta_archivo_salida) as writer:
    DF_estadisticas_costos.to_excel(writer, sheet_name='A1', index=True)
    DF_resumen.to_excel(writer, sheet_name='A2', index=False)

print(f"Análisis completado y resultados guardados en '{ruta_archivo_salida}'")


Análisis completado y resultados guardados en 'C:\Users\Administrator\Documents\GitHub\Hackathon2-2024\1. Desarollo\Resultados\Analisis costos.xlsx'


In [2]:
import pandas as pd

# Cargar los datos procesados del DataFrame de estadísticas desde el archivo Excel
DF_estadisticas_costos = pd.read_excel('Analisis de costos_mod.xlsx', sheet_name='A1')

# Filtrar los datos excluyendo aquellos sin registros
DF_estadisticas_filtradas = DF_estadisticas_costos[DF_estadisticas_costos['Clasificacion'] != 'Sin Registros']

# Calcular las 3 clases con más costos para cada centro
top_clases_por_centro_clasif = (
    DF_estadisticas_filtradas.groupby(['Centro', 'Clase', 'Clasificacion'])['Media']
    .sum()
    .groupby(level=0, group_keys=False)
    .nlargest(3)
    .reset_index()
)

# Calcular los 3 centros con más costos para cada subproceso
top_centros_por_subproceso_clasif = (
    DF_estadisticas_filtradas.groupby(['SubProceso', 'Centro', 'Clasificacion'])['Media']
    .sum()
    .groupby(level=0, group_keys=False)
    .nlargest(3)
    .reset_index()
)

# Calcular los 3 subprocesos con más costos para cada proceso
top_subprocesos_por_proceso_clasif = (
    DF_estadisticas_filtradas.groupby(['Proceso', 'SubProceso', 'Clasificacion'])['Media']
    .sum()
    .groupby(level=0, group_keys=False)
    .nlargest(3)
    .reset_index()
)

# Exportar los resultados a un archivo Excel
ruta_archivo_salida = r'C:\Users\Administrator\Documents\GitHub\Hackathon2-2024\1. Desarollo\Resultados\Resumen_Top_Costos_Clasificacion.xlsx'
with pd.ExcelWriter(ruta_archivo_salida) as writer:
    top_clases_por_centro_clasif.to_excel(writer, sheet_name='Top Clases por Centro', index=False)
    top_centros_por_subproceso_clasif.to_excel(writer, sheet_name='Top Centros por SubProceso', index=False)
    top_subprocesos_por_proceso_clasif.to_excel(writer, sheet_name='Top SubProcesos por Proceso', index=False)

print(f"Resumen exportado a '{ruta_archivo_salida}'")


Resumen exportado a 'C:\Users\Administrator\Documents\GitHub\Hackathon2-2024\1. Desarollo\Resultados\Resumen_Top_Costos_Clasificacion.xlsx'
